Carregando o arquivo

In [3]:
import os

# Diretório do dataset
OUT_DIR = './datasets'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)


In [4]:
import requests

# URL do arquivo "goodreads_interactions.csv"
url = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/goodreads_interactions.csv'
output_path = os.path.join(OUT_DIR, 'goodreads_interactions.csv')

# Função para baixar o arquivo
def download_file(url, local_filename):
    print(f"Baixando {local_filename}...")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"Download concluído: {local_filename}")

# Baixando o arquivo
download_file(url, output_path)


Baixando ./datasets/goodreads_interactions.csv...
Download concluído: ./datasets/goodreads_interactions.csv


Divide a leitura do dataset em chunks com 1 milhão de linhas

In [5]:
import pandas as pd

# Carrega o arquivo em partes
chunksize = 10**6  # 1 milhão de linhas
interactions_iter = pd.read_csv(output_path, chunksize=chunksize)

# Processa cada parte individualmente
for i, chunk in enumerate(interactions_iter):
    print(f"Processando chunk {i+1}")
    print(chunk.head())  # Exibe as primeiras linhas de cada chunk


Processando chunk 1
   user_id  book_id  is_read  rating  is_reviewed
0        0      948        1       5            0
1        0      947        1       5            1
2        0      946        1       5            0
3        0      945        1       5            0
4        0      944        1       5            0
Processando chunk 2
         user_id  book_id  is_read  rating  is_reviewed
1000000     1970    42858        0       0            0
1000001     1970    42807        0       0            0
1000002     1970   302273        0       0            0
1000003     1970   150889        0       0            0
1000004     1970   302272        0       0            0
Processando chunk 3
         user_id  book_id  is_read  rating  is_reviewed
2000000     4037   460415        0       0            0
2000001     4037   460414        0       0            0
2000002     4037   175643        0       0            0
2000003     4037   460413        0       0            0
2000004     4037   46041

Verifica valores ausentes e duplicatas

In [8]:
import pandas as pd

file_path = 'datasets/goodreads_interactions.csv'
chunk_size = 100000  # Ajuste do tamanho do chunk conforme necessário

# Inicializa contadores para valores ausentes e duplicatas
missing_counts = None
duplicate_count = 0

# Itera sobre cada chunk
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Conta valores ausentes em cada coluna do chunk atual
    chunk_missing = chunk.isna().sum()

    # Acumula as contagens de valores ausentes globalmente
    if missing_counts is None:
        missing_counts = chunk_missing
    else:
        missing_counts += chunk_missing

    # Conta duplicatas considerando 'user_id' e 'book_id' como identificadores únicos
    duplicate_count += chunk.duplicated(subset=['user_id', 'book_id']).sum()

# Exibe o total de valores ausentes por coluna e duplicatas no dataset completo
print("Contagem total de valores ausentes por coluna:")
print(missing_counts)
print("\nTotal de duplicatas no dataset:", duplicate_count)


Contagem total de valores ausentes por coluna:
user_id        0
book_id        0
is_read        0
rating         0
is_reviewed    0
dtype: int64

Total de duplicatas no dataset: 0


Percebe-se a falta de valores ausentes e de duplicatas